In [2]:
from google.colab import drive
import warnings
import json
import pandas as pd
import os
import numpy as np

warnings.filterwarnings("ignore")
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Desys_Group/data/'

Mounted at /content/drive


In [4]:
from sklearn.preprocessing import OneHotEncoder

train = pd.read_csv(root+'train.csv')[:10000]
valid = pd.read_csv(root+'valid.csv')[:10000]
test = pd.read_csv(root+'test.csv')[:10000]

encoder = OneHotEncoder() 

encoder_df = pd.DataFrame(encoder.fit_transform(train[['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last']]).toarray())
train = train.join(encoder_df)
train = train.drop(columns=['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last'])
train.replace([np.inf, -np.inf], np.nan, inplace=True)
train.dropna(inplace=True)
train_label = train['label']
train = train.drop(columns=['label'])

encoder_df = pd.DataFrame(encoder.fit_transform(valid[['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last']]).toarray())
valid = valid.join(encoder_df)
valid = valid.drop(columns=['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last'])
valid.replace([np.inf, -np.inf], np.nan, inplace=True)
valid.dropna(inplace=True)
valid_label = valid['label']
valid = valid.drop(columns=['label'])

encoder_df = pd.DataFrame(encoder.fit_transform(test[['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last']]).toarray())
test = test.join(encoder_df)
test = test.drop(columns=['to_addr', 'collection_slug', 'contract_type_last', 'payment_type_last'])
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test.dropna(inplace=True)
test_label = test['label']
test = test.drop(columns=['label'])

## LR

In [13]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

LR = LogisticRegression(random_state = 42, tol=0.1, max_iter=1)
LR.fit(train, train_label)

pred_prob_LR = LR.predict_proba(test)[::,1]
AUC_LR = metrics.roc_auc_score(test_label, pred_prob_LR)
print("AUC of LR:", AUC_LR)

logloss_LR = metrics.log_loss(test_label, LR.predict_proba(test))
print("Logloss of LR", logloss_LR)

AUC of LR: 0.9708124732522481
Logloss of LR 0.32755196774306544


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

NB = GaussianNB(var_smoothing=10)
NB.fit(train, train_label)

pred_prob_NB = NB.predict_proba(test)[::,1]
AUC_NB = metrics.roc_auc_score(test_label, pred_prob_NB)
print("AUC of NB:", AUC_NB)

logloss_NB = metrics.log_loss(test_label, NB.predict_proba(test))
print("Logloss of NB", logloss_NB)

AUC of NB: 0.9708124732522481
Logloss of NB 0.4641925973454172


## Tree

In [7]:
from sklearn import tree

tree = tree.DecisionTreeClassifier(max_depth=1,max_leaf_nodes=2)
tree.fit(train, train_label)

pred_prob_tree = tree.predict_proba(test)[::,1]
AUC_tree = metrics.roc_auc_score(test_label, pred_prob_tree)
print("AUC of tree:", AUC_tree)

logloss_tree = metrics.log_loss(test_label, tree.predict_proba(test))
print("Logloss of tree", logloss_tree)

AUC of tree: 0.936084331705044
Logloss of tree 0.1908515923576755


## Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(max_depth=2)
RF.fit(train, train_label)

pred_prob_RF = RF.predict_proba(test)[::,1]
AUC_RF = metrics.roc_auc_score(test_label, pred_prob_RF)
print("AUC of RF:", AUC_RF)

logloss_RF = metrics.log_loss(test_label, RF.predict_proba(test))
print("Logloss of RF", logloss_RF)

AUC of RF: 0.9429943680946483
Logloss of RF 0.47478070019885643
